In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

%matplotlib
matplotlib.rcParams.update({'font.size': 20})

Using matplotlib backend: TkAgg


In [4]:
import scipy.fftpack
from scipy.signal import gaussian

In [5]:
def f_beta(a,q,beta_guess):
    return q**2/( (beta_guess+2)**2 - a - q**2/( (beta_guess+4)**2 - a - q**2/( (beta_guess+6)**2 - a - q**2/( (beta_guess+8)**2 - a - q**2/( (beta_guess+10)**2 - a - q**2/( (beta_guess+10)**2 - a - q**2/( (beta_guess+12)**2 - a ) ) ) ) ) ) )

def beta_continue_alamano(a,q,beta_guess): #a l'air ok
    return a + f_beta(a,q,beta_guess) + f_beta(a,q,-beta_guess)

# GiantMol

In [64]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide


r0 = 2.865e-3/1.14511 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+
Omega = 2.0e6*2*pi # 2.047e6
bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
# wzLC = (2*pi*100000)**2
wzLC = 2*pi*90806.9982303
K = 1e5*2*pi
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.000000


In [7]:
q = np.linspace(0.6,0.7,11)
print(q)

[0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7 ]


In [11]:
# q and alpha=R/L known
# compute ax Udc
q = 0.60
print('q',q)
Urf = q*(m_Ca*r0**2*Omega**2)/(4*C_e)
print('Urf =',Urf)
beta_guess = 0.423
ax = 0
for i in range(50):
    for k in range(50):
        beta_guess = beta_continue_alamano(ax,q,beta_guess)
        beta_guess = sqrt(beta_guess)
    omega_r_2 = ( beta_guess*Omega/2 )**2
    ax = -2*omega_r_2/Omega**2
# omega_z_2 = omega_r_2
Udc_for_alpha1 = omega_r_2/K**2
print('ax =',ax)
print('beta =',beta_guess)
# Udc_for_alpha1 = omega_r_2/K**2
# # omega_z_2 = K**2 * Udc_for_alpha1
# omega_x_2 = omega_r_2 + 0.5*omega_z_2
print('Udc =',Udc_for_alpha1)
print(omega_z_2/omega_r_2)

q 0.6
Urf = 61.596877310090996
ax = -0.06541616226969633
beta = 0.361707512417689
Udc = 13.083232453939267


NameError: name 'omega_z_2' is not defined

In [12]:
# Udc and Urf known
Urf = 61.597
q = 0.6
Udc = 13.083232453939267

omega_z_2 = K**2*Udc

beta_guess = 0.423
ax = -2/Omega**2*omega_z_2
for i in range(50):
    beta_guess = beta_continue_alamano(ax,q,beta_guess)
    beta_guess = sqrt(beta_guess)
omega_r_2 = ( beta_guess*Omega/2 )**2

print(sqrt(omega_z_2)/2/pi)
print(sqrt(omega_r_2)/2/pi)
print(omega_z_2/omega_r_2)

361707.512417689
361707.51243116683
0.9999999999254766


In [65]:
figure('Diagramme de stabilité').clf()

U_rf = linspace(0,100,1001)
q = 4*C_e*U_rf/(m_Ca*r0**2*Omega**2)

a0 = -1/2*q**2 + 7/128*q**4 - 29/2304*q**6  + 68687/18874368*q**8
b1 = 1 - q - q**2/8 + q**3/64 - q**4/1536 - 11/36864*q**5 + 49/589824*q**6 - 55/9437184*q**7 - 83/35389440*q**8

plot(U_rf,a0,color='tab:purple',label='Ca$^+$')
plot(U_rf,b1,color='tab:purple')
plot(U_rf,-a0,color='tab:purple')
plot(U_rf,-b1,color='tab:purple')

grid()
legend()
xlabel('$U_{RF}$ [V]')
ylabel('a')
xlim(-0.1,100)
ylim(-0.55,0.55)
tight_layout()

In [63]:
wzLC

325535515128.0366

In [128]:
Urf = linspace(0,100,1000)
qx = 4*C_e*Urf/(m_Ca*r0**2*Omega**2)
Udc_test = 2.5
omega_z_2 = wzLC**2*Udc_test
au = -2/Omega**2*omega_z_2

beta_guess_na = 0.423
for k in range(15):
    beta_guess_na = beta_continue_alamano(au,qx,beta_guess_na)
    beta_guess_na = sqrt(beta_guess_na)
omega_r_2_na = ( beta_guess_na*Omega/2 )**2

q_base = linspace(0,0.9,10)
Urf_q = q_base*(m_Ca*r0**2*Omega**2)/( 4*C_e)
beta_guess_base = 0.423
for k in range(15):
    beta_guess_base  = beta_continue_alamano(au,q_base,beta_guess_base)
    beta_guess_base = sqrt(beta_guess_base)
omega_r_2_base = ( beta_guess_base*Omega/2 )**2

/tmp/ipykernel_21381/512395964.py:10: RuntimeWarning: invalid value encountered in sqrt
  beta_guess_na = sqrt(beta_guess_na)
/tmp/ipykernel_21381/512395964.py:18: RuntimeWarning: invalid value encountered in sqrt
  beta_guess_base = sqrt(beta_guess_base)


In [129]:
figure('Plot radial freq').clf()
ax = plt.subplot(111)

ax.plot(Urf,sqrt(omega_r_2_na)/2/pi*1e-3,color='xkcd:bright orange',label=r'$f_r=\frac{\Omega}{2}\beta \frac{1}{2\pi}$ (non adiabatique)')
ax.plot(Urf,sqrt(au+qx**2/2)*Omega/2/2/pi*1e-3,color='k',ls=':',label=r'$f_r = \sqrt{\frac{1}{2}q_u^2+a_u}\frac{1}{2\pi}$ (adiabatique)')
ax.plot(Urf[50::100],sqrt(omega_r_2_na[50::100])/2/pi*1e-3,'P',color='xkcd:azul')
ax.plot(Urf_q,sqrt(omega_r_2_base)/2/pi*1e-3,'P',color='xkcd:brown')

for i,j in enumerate(sqrt(omega_r_2_na[50::100])/2/pi*1e-3):
    t=plt.text(Urf[50::100][i]-1.5, j+20, f'$f_r=${j:.1f}\n$q_u$={qx[50::100][i]:.2f}',
               transform=ax.transData, ha='right',
               fontsize=13,color='xkcd:azul')
    t.set_bbox(dict(facecolor='white', alpha=0.8, edgecolor='k'))
    #text(Urf[::200][i]-3.2,j+21,f'{j:.1f}',fontsize=12,color='xkcd:azul')
for i,j in enumerate(sqrt(omega_r_2_base)/2/pi*1e-3):
    t=plt.text(Urf_q[i]+2, j-65, f'$f_r=${sqrt(omega_r_2_base[i])/2/pi*1e-3:.1f}\n$q_u=${q_base[i]:.1f}',
               transform=ax.transData, ha='left',
               fontsize=13,color='xkcd:brown')
    t.set_bbox(dict(facecolor='white', alpha=0.8, edgecolor='k'))    
    
title(r'$f_r$ vs $U_{RF}$')
ax.set_xlabel('$U_{RF}$ [V] (4$U_{RF}$ [V] = $U_{RF}$ [V$_{pp}$])')
ax.set_ylabel(r'$f_r$ [kHz]')
ax.grid(visible=True, which='major', color='0.2', linestyle='-')
ax.grid(visible=True, which='minor', color='0.7', linestyle='--')
plt.minorticks_on()

t = plt.text(0.025, 0.8, f'$\Omega_{{RF}}$ = {Omega/2/pi/1e6:.2f}$\\times 2\pi$ MHz\n$U_{{DC}} = {Udc_test:.3f}$ V\n$f_z  ={sqrt(omega_z_2)/2/pi*1e-3:.1f}$ kHz\n$U_{{RF}}$ min = {Urf[nanargmin(sqrt(omega_r_2_na)/2/pi*1e-3)]:.1f} V', transform=ax.transAxes, fontsize=20)
t.set_bbox(dict(facecolor='white', alpha=0.8, edgecolor='k'))

ax.set_xlim(-1,103)
ax.set_ylim(-75,1350)

plt.legend(loc=4,fontsize = 13)
plt.tight_layout()

savefig('fr_vs_Urf_Udc2.5V.png',dpi=300)

/tmp/ipykernel_21381/443910433.py:5: RuntimeWarning: invalid value encountered in sqrt
  ax.plot(Urf,sqrt(au+qx**2/2)*Omega/2/2/pi*1e-3,color='k',ls=':',label=r'$f_r = \sqrt{\frac{1}{2}q_u^2+a_u}\frac{1}{2\pi}$ (adiabatique)')
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


In [117]:
Udc

array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. ,
       6.5, 7. , 7.5, 8. , 8.5, 9. , 9.5])

# TADOTI

In [ ]:
r0=3.93e-3 # 2.5e-3
d0=21e-2/2

Omega=5.215e6*2*pi # 2.047e6

mkappa = 9.6*1e-4

In [ ]:
Udc = 1200
Urf = 830/4
omega_z_2 = 2*C_e*mkappa*Udc/m_Ca/d0**2
omega_x_2 = 2*(C_e*Urf/m_Ca/Omega/r0**2)**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2

qx = 4*C_e*Urf/(m_Ca*r0**2*Omega**2)
az = 2*omega_z_2/Omega**2

print('alpha =',omega_z_2/omega_x_2)
print('qx =',qx)
print('ax =',ax)
print('az =',az)
print('f_z =',sqrt(omega_z_2)/2/pi)
print('f_x =',sqrt(omega_x_2)/2/pi)
print('f_r =',sqrt(omega_r_2)/2/pi)